In [1]:
import itertools
import sys, os

import numpy as np
import pandas as pd
from scipy.special import comb
from scipy import stats
import scipy.cluster.hierarchy as hac
import matplotlib.pyplot as plt
plt.style.use('classic')
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns

from collections import Counter
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

sns.set(rc={'figure.figsize':(15,8)})
sns.set_context('poster')

In [2]:
%load_ext autoreload
%autoreload 2
#import CCPA_lib as cp
import sorf_lib as sf

In [3]:
sns.set_context('poster')

Prochlorococcus marinus str. MIT 9313, complete genome
https://www.ncbi.nlm.nih.gov/nuccore/NC_005071.1/

    
Prochlorococcus marinus MIT9313 complete genome
https://www.ncbi.nlm.nih.gov/nuccore/BX548175




![](http://oregonstate.edu/instruct/bb450/fall14/stryer7/2/table_02_02.jpg)

https://www.uniprot.org/uniprot/Q7V735

http://tigrfams.jcvi.org/cgi-bin/HmmReportPage.cgi?acc=TIGR03798

https://www.ebi.ac.uk/training/online/course/interpro-functional-and-structural-analysis-protei/sequence-searching/searching-interpro-batc

http://www.ebi.ac.uk/interpro/sequencesearch/iprscan5-S20190707-131508-0462-76111813-p1m

https://www.ebi.ac.uk/Tools/services/rest/iprscan5/result/iprscan5-S20190707-131508-0462-76111813-p1m/json

In [4]:
genome='MIT9313'

In [5]:
import os
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.SeqFeature import SeqFeature, FeatureLocation
from Bio import SeqIO

In [6]:
# get all sequence records for the specified genbank file
recs = [rec for rec in SeqIO.parse(os.path.join('data', "MIT9313.gb"), "genbank")]

# print the number of sequence records that were extracted
len(recs)


1

In [7]:
int(recs[0].features[1].location.end)

1340

In [8]:
    merge_cols = ['contig_id', 'gene_id', 'location', 'type', 
                  'start', 'stop', 'strand',
                  'function', 'figfam',
                  'nucleotide_sequence', 'aa_sequence', 'genome', 'left', 'right']


In [9]:

def gb_to_record(i, genome='MIT9313'):
    q = i.qualifiers
    g = lambda x : ','.join(q.get(x, []))
    pmtid = [i for i in q.get('old_locus_tag',[]) if i.startswith('PMT')]
    record = {
        'contig_id' : genome, 
        'gene_id' : g('locus_tag'), 
        'pmt_id': ','.join(pmtid), 
        'type' : i.type, 
        'location' : g('locus_tag'),  
        'strand' : '+' if i.location.strand == 1 else '-',
        'start' : int(i.location.start),
        'stop' : int(i.location.end),
        'left' : int(i.location.start),
        'right' : int(i.location.end),
        'function' : g('product'), 
        'genome' : genome, 
        'old_locus_tag' : g('old_locus_tag'), 
        'product' : g('product'), 
        'db_xref' : g('db_xref'), 
        'protein_id' : g('protein_id'), 
        'figfam' : '',
        'nucleotide_sequence' : '', 
        'aa_sequence' : g('translation'), 
    }
    return record


In [10]:
types_to_collect = ['CDS', 'ncRNA', 'rRNA', 'regulatory', 'tRNA', 'tmRNA']
records = [gb_to_record(i) for i in recs[0].features if i.type in types_to_collect]

In [11]:
gdf = pd.DataFrame(records)

In [13]:

gdf.to_pickle(os.path.join('data', f'{genome}.ncbi.gb.pkl.gz'))


In [14]:
gdf.loc[gdf.type != 'CDS'].head(20)

,contig_id,gene_id,pmt_id,type,location,strand,start,stop,left,right,function,genome,old_locus_tag,product,db_xref,protein_id,figfam,nucleotide_sequence,aa_sequence
44,MIT9313,AKG35_RS00225,PMT_RNA_44,tRNA,AKG35_RS00225,-,42281,42353,42281,42353,tRNA-Gly,MIT9313,"PMT_RNA_44,RG24_RS00225,RNA_44",tRNA-Gly,,,,,
84,MIT9313,AKG35_RS00430,PMT_RNA_1,tRNA,AKG35_RS00430,+,87819,87891,87819,87891,tRNA-Gly,MIT9313,"PMT_RNA_1,RG24_RS00430,RNA_1",tRNA-Gly,,,,,
162,MIT9313,,,regulatory,,+,178355,178459,178355,178459,,MIT9313,,,RFAM:RF00059,,,,
187,MIT9313,AKG35_RS00920,,rRNA,AKG35_RS00920,+,208843,210328,208843,210328,16S ribosomal RNA,MIT9313,"RG24_RS00925,RNA_53",16S ribosomal RNA,RFAM:RF00177,,,,
188,MIT9313,AKG35_RS00925,PMT_RNA_2,tRNA,AKG35_RS00925,+,210510,210584,210510,210584,tRNA-Ile,MIT9313,"PMT_RNA_2,RG24_RS00930,RNA_2",tRNA-Ile,,,,,
189,MIT9313,AKG35_RS00930,PMT_RNA_3,tRNA,AKG35_RS00930,+,210593,210666,210593,210666,tRNA-Ala,MIT9313,"PMT_RNA_3,RG24_RS00935,RNA_3",tRNA-Ala,,,,,
190,MIT9313,AKG35_RS00940,,rRNA,AKG35_RS00940,+,211157,214034,211157,214034,23S ribosomal RNA,MIT9313,"RG24_RS00945,RNA_54",23S ribosomal RNA,RFAM:RF02541,,,,
191,MIT9313,AKG35_RS00945,,rRNA,AKG35_RS00945,+,214141,214258,214141,214258,5S ribosomal RNA,MIT9313,"RG24_RS00950,RNA_55",5S ribosomal RNA,RFAM:RF00001,,,,
207,MIT9313,AKG35_RS01015,PMT_RNA_43,tRNA,AKG35_RS01015,-,227744,227816,227744,227816,tRNA-Thr,MIT9313,"PMT_RNA_43,RG24_RS01020,RNA_43",tRNA-Thr,,,,,
208,MIT9313,AKG35_RS01020,PMT_RNA_42,tRNA,AKG35_RS01020,-,227826,227908,227826,227908,tRNA-Tyr,MIT9313,"PMT_RNA_42,RG24_RS01025,RNA_42",tRNA-Tyr,,,,,


In [15]:
genome = 'MIT9313'
accession = sf.get_accession(genome)
accession

'BX548175'

In [16]:
orf_dpath = 'orffinder_tmp'
orf_fpath = os.path.join(orf_dpath, f'{accession}.orffinder.fa')
df = sf.parse_orffinder_fasta(orf_fpath)

In [17]:
df['rast_left'] = df['left']
df['rast_right'] = df['right'] + 1

In [18]:
gdf.loc[gdf.location == 'AKG35_RS00020']

,contig_id,gene_id,pmt_id,type,location,strand,start,stop,left,right,function,genome,old_locus_tag,product,db_xref,protein_id,figfam,nucleotide_sequence,aa_sequence
3,MIT9313,AKG35_RS00020,"PMT0004,PMT_0004",CDS,AKG35_RS00020,+,4621,6079,4621,6079,amidophosphoribosyltransferase,MIT9313,"PMT0004,PMT_0004,RG24_RS00020",amidophosphoribosyltransferase,,WP_011129383.1,,,MCGIVGIVSTALVNQQIYDSLLLLQHRGQDSTGIATMDGSVFHLHK...


In [19]:
df.loc[(df.rast_left > 4620) & (df.rast_left < 4630),  ].T

,5047
orfid,ORF5048_BX548175.1:4621:6078
contig,BX548175.1
left,4621
right,6078
start,4621
stop,6078
strand,+
aaseq,MCGIVGIVSTALVNQQIYDSLLLLQHRGQDSTGIATMDGSVFHLHK...
len_nn,1458
len_aa,485


In [20]:
df = sf.add_gene_annotation_to_sorf_df(df, gdf)
df = sf.add_overlapping_genes(df, gdf)

In [23]:
summary_df = sf.gen_summary_df(df, genome)
df.to_pickle(os.path.join(orf_dpath, f'{genome}.ncbi.sorf.pkl.gz'))
summary_df.to_csv(os.path.join(orf_dpath, f'{genome}.ncbi.sorf.summary.csv.gz'))


In [ ]:
summary_df

In [ ]:
orfid = 'ORF16879_BX548175.1:1658121:1657948'
orfid = 'ORF19962_BX548175.1:317208:317134'

In [ ]:
df.loc[df.orfid.isin([orfid])].T

In [ ]:
gdf.columns

In [ ]:
gdf.loc[(gdf.left > 317100) & (gdf.right < 317220)]

In [ ]:
gdf.loc[gdf['product'].str.contains('hypothetical protein')]

In [ ]:
gdf.loc[gdf.protein_id.str.contains('WP_011129965')]

In [ ]:
df.loc[df.orfid== 'ORF30023_BX548175.1:644083:643928']

In [ ]:
summary_df

In [ ]:
gdf.left.value_counts()

In [ ]:
gdf.head()

In [32]:
gdf.type.value_counts()

CDS           2403
tRNA            44
rRNA             6
ncRNA            3
regulatory       2
tmRNA            1
Name: type, dtype: int64

In [34]:
df.loc[df.orfid == 'ORF3795_BX548175.1:1742715:1742936',
[
    'left', 'left_overlap', 'right', 'right_overlap', 'rast_right', 'rast_left','overlap_type' ,
    'start', 'start_overlap', 'strand',
    'aaseq', 'aa_sequence_overlap', 'otype'
]
                  
                 ].T

,3794
left,1742715
left_overlap,NaN
right,1742936
right_overlap,NaN
rast_right,1742937
rast_left,1742715
overlap_type,upstream
start,1742715
start_overlap,NaN
strand,+
